In [1]:
pip install nltk

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install spacy


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


import libraries

In [14]:
#import libraries
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import spacy
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package punkt to /Users/henry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/henry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


load dataset

In [15]:
# Load the dataset
data = pd.read_csv('Twitter Sentiment/dataset.csv', header=None)
data.columns = ['labels', 'text']
print(len(data))
print(data.head())

100000
   labels                                               text
0       0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1       0  is upset that he can't update his Facebook by ...
2       0  @Kenichan I dived many times for the ball. Man...
3       0    my whole body feels itchy and like its on fire 
4       0  @nationwideclass no, it's not behaving at all....


Data preprocessing

We clean the data by removing url mentions hashtags and keeping only the letters then convert to lower case

Then we tokenise the text then we remove stop words and lanel the data

In [16]:
# Clean and preprocess the text data
import spacy

def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'[^A-Za-z]+', ' ', text)  # Keep only letters
    text = text.lower()  # Convert to lowercase
    return text

data['cleaned_text'] = data['text'].apply(clean_text)

# Tokenize the text
data['tokens'] = data['cleaned_text'].apply(word_tokenize)

# Stemming the individual words
stemmer = PorterStemmer()

def stem_words(word_list):
    return [stemmer.stem(word) for word in word_list]

data['stemmed_tokens'] = data['tokens'].apply(stem_words)

# Lemmatizating the individual words 

nlp = spacy.load("en_core_web_sm")

def lemmatize_words(word_list):
    doc = nlp(" ".join(word_list))  # Join the stemmed tokens into a string
    lemmatized_text = [token.lemma_ for token in doc]
    return lemmatized_text

data['lemmatized_token'] = data['stemmed_tokens'].apply(lemmatize_words)

# Remove stop words
stop_words = set(stopwords.words('english'))
#data['filtered_tokens'] = data['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

data.head()

,labels,text,cleaned_text,tokens,stemmed_tokens,lemmatized_token
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww that s a bummer you shoulda got david ca...,"[awww, that, s, a, bummer, you, shoulda, got, ...","[awww, that, s, a, bummer, you, shoulda, got, ...","[awww, that, s, a, bummer, you, shoulda, get, ..."
1,0,is upset that he can't update his Facebook by ...,is upset that he can t update his facebook by ...,"[is, upset, that, he, can, t, update, his, fac...","[is, upset, that, he, can, t, updat, hi, faceb...","[be, upset, that, he, can, t, updat, hi, faceb..."
2,0,@Kenichan I dived many times for the ball. Man...,i dived many times for the ball managed to sa...,"[i, dived, many, times, for, the, ball, manage...","[i, dive, mani, time, for, the, ball, manag, t...","[I, dive, mani, time, for, the, ball, manag, t..."
3,0,my whole body feels itchy and like its on fire,my whole body feels itchy and like its on fire,"[my, whole, body, feels, itchy, and, like, its...","[my, whole, bodi, feel, itchi, and, like, it, ...","[my, whole, bodi, feel, itchi, and, like, it, ..."
4,0,"@nationwideclass no, it's not behaving at all....",no it s not behaving at all i m mad why am i ...,"[no, it, s, not, behaving, at, all, i, m, mad,...","[no, it, s, not, behav, at, all, i, m, mad, wh...","[no, it, s, not, behav, at, all, I, m, mad, wh..."


In [12]:
stop_words = set(stopwords.words('english'))
data['filtered_tokens'] = data['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

data.head()


KeyError: 'tokens'

In [11]:
# One hot encoding

categories = list(set([item for sublist in data['filtered_tokens'] for item in sublist]))
print(categories)
'''vectorizer = CountVectorizer(binary=True, vocabulary=categories)
sentences = [' '.join(words) for words in data['filtered_tokens']]

# Fit and transform the sentences
one_hot_encoded = vectorizer.fit_transform(sentences)

# Convert the one-hot encoded data to a DataFrame
encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns = categories)
#encoded_df['labels'] = data['labels']

# use the print function to check if the one hot encoding is correct
#print(encoded_df.iloc[954,:].sum(),sentences[954]) 
print(encoded_df)'''



KeyError: 'filtered_tokens'

In [59]:
count = (data['labels'] == 0).sum()
print(count)


5000


In [60]:
# Split the dataset

X_train, X_test, y_train, y_test=train_test_split(encoded_df, data['labels'], stratify=data['labels'], test_size=0.2, random_state=42)



In [61]:
print(y_train)
sum(y_train)

2858     0
96842    1
3905     0
97684    1
1564     0
        ..
99987    1
1717     0
98795    1
3691     0
131      0
Name: labels, Length: 8000, dtype: int64


4000

then we do bigram analysos

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an N-gram model (e.g., bigrams)
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))
X_train_ngrams = ngram_vectorizer.fit_transform(X_train)
X_test_ngrams = ngram_vectorizer.transform(X_test)


use a multinomial nb classifier

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Train a classifier (e.g., Multinomial Naive Bayes)
classifier = MultinomialNB()
classifier.fit(X_train_ngrams, Y_train)

# Make predictions
y_pred = classifier.predict(X_test_ngrams)

# Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
classification_rep = classification_report(Y_test, y_pred, target_names=le.classes_)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')


ValueError: Found input variables with inconsistent numbers of samples: [2791, 800]

In [ ]:
#test